## 4. Comprehensions and Generators

### 29 Avoid Repeated Work in Comprehensions by Using Assignment Expressions

In [1]:
import logging

In [2]:
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

In [3]:
result = {}
for name in order:
  count = stock.get(name, 0)
  batches = get_batches(count, 8)
  if batches:
    result[name] = batches

print(result)

{'screws': 4, 'wingnuts': 1}


In [4]:
found = {name: get_batches(stock.get(name, 0), 8)
         for name in order
         if get_batches(stock.get(name, 0), 8)}
print(found)

{'screws': 4, 'wingnuts': 1}


In [5]:
has_bug = {name: get_batches(stock.get(name, 0), 4)
           for name in order
           if get_batches(stock.get(name, 0), 8)}

print('Expected:', found)
print('Found:   ', has_bug)

Expected: {'screws': 4, 'wingnuts': 1}
Found:    {'screws': 8, 'wingnuts': 2}


In [6]:
found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}
assert found == {'screws': 4, 'wingnuts': 1}, found

In [7]:
try:
    result = {name: (tenth := count // 10)
              for name, count in stock.items() if tenth > 0}
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-7-a68c08c51946>", line 2, in <module>
    result = {name: (tenth := count // 10)
  File "<ipython-input-7-a68c08c51946>", line 3, in <dictcomp>
    for name, count in stock.items() if tenth > 0}
NameError: name 'tenth' is not defined


In [8]:
result = {name: tenth for name, count in stock.items()
          if (tenth := count // 10) > 0}
print(result)

{'nails': 12, 'screws': 3, 'washers': 2}


In [9]:
half = [(last := count // 2) for count in stock.values()]
print(f'Last item of {half} is {last}')

Last item of [62, 17, 4, 12] is 12


In [10]:
for count in stock.values():  # Leaks loop variable
    pass
print(f'Last item of {list(stock.values())} is {count}')

Last item of [125, 35, 8, 24] is 24


In [11]:
try:
    del count
    half = [count // 2 for count in stock.values()]
    print(half)   # Works
    print(count)  # Exception because loop variable didn't leak
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-11-595a79eac01a>", line 5, in <module>
    print(count)  # Exception because loop variable didn't leak
NameError: name 'count' is not defined


[62, 17, 4, 12]


In [12]:
found = ((name, batches) for name in order
         if (batches := get_batches(stock.get(name, 0), 8)))
print(next(found))
print(next(found))

('screws', 4)
('wingnuts', 1)


> - 대입식을 통해 컴프리헨션이나 제너레이터 식의 조건 부분에서 사용한 값을 같은 컴프리헨션이나 제너레이터의 다른 위치에서 재사용할 수 있다. 이를 통해 가독성과 성능을 향상시킬 수 있다.
> - 조건이 아닌 부분에도 대입식을 사용할 수 있지만, 그런 형태의 사용은 피해야 한다.